# 곱하기 노드 

In [2]:
class Multilayer:
    def __init__ (self):
        self.x=None
        self.y=None

    def forward(self,x,y):
        self.x=x
        self.y=y
        out=x*y
        return out 

    def backward(self,dout)  :
        dx=dout*self.y
        dy=dout*self.x
        return dx,dy

## 순전파

In [7]:
apple=100
apple_num=2
tax=1.1

In [8]:
price_layer=Multilayer()
tax_layer=Multilayer()

In [9]:
apple_price=price_layer.forward(apple,apple_num)
print('apple_price : ',apple_price)
final_price=tax_layer.forward(apple_price,tax)
print('final_price : ',final_price)

apple_price :  200
final_price :  220.00000000000003


## 역전파

In [10]:
dout=1
dapple_price,dtax=tax_layer.backward(dout)
print(f'dapple_price : {dapple_price}')
print(f'dtax : {dtax}')

dapple,dapple_num=price_layer.backward(dapple_price)
print(f'dapple : {dapple}')
print(f'dapple_num : {dapple_num}')

dapple_price : 1.1
dtax : 200
dapple : 2.2
dapple_num : 110.00000000000001


# 더하기 노드

In [11]:
class Addlayer:
    def __init__(self):
        pass

    def forward(self,x,y):
        out=x+y
        return out

    def backward(self,dout):
        dx=dout*1
        dy=dout*1
        return dx,dy

# Apple + Orange  

In [12]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

In [15]:
apple_layer=Multilayer()
orange_layer=Multilayer()
sum_layer=Addlayer()
tax_layer=Multilayer()

## 순전파

In [16]:
apple_price=apple_layer.forward(apple,apple_num)
orange_price=orange_layer.forward(orange,orange_num)

together_price=sum_layer.forward(apple_price,orange_price)

final_price=tax_layer.forward(together_price,tax)

print(f'apple_price : {apple_price}')
print(f'orange_price : {orange_price}')
print(f'together_price : {together_price}')
print(f'final_price : {final_price}')

apple_price : 200
orange_price : 450
together_price : 650
final_price : 715.0000000000001


## 역전파

In [17]:
dout=1
dtogether_price,dtax=tax_layer.backward(dout)
dapple_price,dorange_price=sum_layer.backward(dtogether_price)
dapple,dapple_num=apple_layer.backward(dapple_price)
dorange,dorange_num=orange_layer.backward(dorange_price)

In [18]:
print(f'dtogether_price : {dtogether_price}')
print(f'dtax : {dtax}')
print()
print(f'dapple_price : {dapple_price}')
print(f'dorange_price : {dorange_price}')
print()
print(f'dapple : {dapple}')
print(f'dapple_num : {dapple_num}')
print()
print(f'dorange : {dorange}')
print(f'dorange_num : {dorange_num}')

dtogether_price : 1.1
dtax : 650

dapple_price : 1.1
dorange_price : 1.1

dapple : 2.2
dapple_num : 110.00000000000001

dorange : 3.3000000000000003
dorange_num : 165.0


---  
# Activation Function : __ReLU__  
$y=\begin{cases}x & (x>0)\\
0 & (x\leq0)\end{cases}$  
<br/>  

$\frac{\partial y}{\partial x}=\begin{cases}1 & (x>0)\\
0 & (x\leq 0) \end{cases}$

In [19]:
import numpy as np

In [20]:
class relu:
    def __init__(self):
        self.mask=None

    def forward(self,x):
        self.mask=(x<=0)
        out=x.copy()
        out[self.mask]=0 # 음수인 부분만 0으로 바꾸기
        return out

    def backward(self,dout):
        dout[self.mask]=0
        dx=dout
        return dout

In [21]:
relu=relu()

In [22]:
x=np.array([[1.0,-1.4],
            [-2.0,3.0]])
relu.forward(x)

array([[1., 0.],
       [0., 3.]])

In [23]:
dout=np.array([[-2.3,4.3],
               [3.3,2.0]])
relu.backward(dout)

array([[-2.3,  0. ],
       [ 0. ,  2. ]])

# 행렬 연산을 위한 지식들..

## shape

In [25]:
X=np.random.rand(2) #1x2
W=np.random.rand(2,3) #2x3
B=np.random.rand(3) #1x3

In [26]:
print(X.shape)
print(W.shape)
print(B.shape)

(2,)
(2, 3)
(3,)


In [27]:
Y=np.dot(X,W)+B

In [28]:
print(Y)
print(Y.shape)

[1.06848548 1.20665292 1.82611574]
(3,)


## Transpose

In [29]:
W

array([[0.69439273, 0.53285249, 0.82444108],
       [0.32888719, 0.74301003, 0.95953457]])

In [30]:
W.T

array([[0.69439273, 0.32888719],
       [0.53285249, 0.74301003],
       [0.82444108, 0.95953457]])

## BroadCast

In [31]:
XW=np.array([[0,0,0],
             [10,10,10]])
B=np.array([1,2,3])

In [32]:
XW+B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [33]:
db=np.sum(XW+B,axis=0)
db

array([12, 14, 16])

# Affine 구현  
__Affine__이란 : 신경망 순전파 때 수행하는 행렬의 곱을 기하학에서 부르는 이름  
즉, $Y=X\cdot W+b$ 연산을 실행하는 노드

In [35]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b

    def forward(self,X):
        self.X=X
        Y=np.dot(self.W,X)+self.b
        return Y

    def backward(self,dout):
        dX=np.dot(dout,self.W.T)
        self.dW=np.dot(self.X.T,dout)
        self.db=np.sum(dout,axis=0)

        return dX

# SoftmaxWithLoss

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
cd /content/drive/MyDrive/AI부트캠프/SeSAC_DL/

/content/drive/MyDrive/AI부트캠프/SeSAC_DL


In [39]:
from common.functions import *

In [40]:
class SoftmaxWithLoss:
    def __init__(self):
        pass

    def forward(self,y_pred,y_true):
        self.y_true=y_true
        self.y_pred=softmax(y_pred)
        self.loss=cross_entropy_error(self.y_pred,self.y_true)

        return self.loss

    def backward(self,dout=1):
        batch_size=self.y_true.shape[0]
        dx=(self.y_pred-self.y_true)/batch_size
        return dx